In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from credentials import FacebookCookies

driver_manager = ChromeDriverManager(latest_release_url="https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.60/linux64/chrome-linux64.zip").install()
driver_service = Service(driver_manager)
driver_options = webdriver.ChromeOptions()
cookies_mngr = FacebookCookies()

chrome = webdriver.Chrome(service=driver_service, options=driver_options)
# chrome.get("https://mbasic.facebook.com")
# for cookie in cookies_mngr.load():
#     chrome.add_cookie(cookie)

chrome.get("https://facebook.com/804109268529640")
orig_tab = chrome.current_window_handle

In [5]:
chrome.find_element(By.XPATH, "//div[@class='x78zum5 xdt5ytf x2lah0s x193iq5w x2bj2ny x1ey2m1c xayqjjm x9f619 xds687c x1xy6bms xn6708d x1s14bel x1ye3gou xixxii4 x17qophe x1u8a7rm']")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='x78zum5 xdt5ytf x2lah0s x193iq5w x2bj2ny x1ey2m1c xayqjjm x9f619 xds687c x1xy6bms xn6708d x1s14bel x1ye3gou xixxii4 x17qophe x1u8a7rm']"}
  (Session info: chrome=126.0.6478.126)
Stacktrace:
#0 0x62f91029dc7a <unknown>
#1 0x62f90ff80e2c <unknown>
#2 0x62f90ffcd661 <unknown>
#3 0x62f90ffcd751 <unknown>
#4 0x62f910011f64 <unknown>
#5 0x62f90fff05ed <unknown>
#6 0x62f91000f303 <unknown>
#7 0x62f90fff0363 <unknown>
#8 0x62f90ffc0247 <unknown>
#9 0x62f90ffc0b9e <unknown>
#10 0x62f91026424b <unknown>
#11 0x62f9102682f1 <unknown>
#12 0x62f91024fafe <unknown>
#13 0x62f910268e52 <unknown>
#14 0x62f91023479f <unknown>
#15 0x62f91028d638 <unknown>
#16 0x62f91028d810 <unknown>
#17 0x62f91029cdac <unknown>
#18 0x7736eb49ca94 <unknown>
#19 0x7736eb529c3c <unknown>


In [32]:
import pandas as pd

df = pd.read_csv("kltn/BeatvnNow/BeatvnNow.csv")
pd.DataFrame(df.groupby(["page_id", "type"]).count().loc[:, "post_id"]).rename(columns={"post_id": "count"})

count
page_id   type          
BeatvnNow comment    132
          post        13

In [31]:
df

,page_id,post_id,post_url,cmt_id,cmt_url,datetime,text,images,type
0,BeatvnNow,811181451155755,https://facebook.com/811181451155755,NaN,NaN,2024-07-03 20:54:47.962130,🔥ĐI NHẬT “CHỮA LÀNH” NGAY TRONG TẦM TAY🔥\nTới ...,811181451155755__0.jpg 811181451155755__1.jp...,post
1,BeatvnNow,810483054558928,https://facebook.com/810483054558928,NaN,NaN,2024-07-02 22:56:48.184952,"NỬA ĐẦU NĂM NAY, HÀ NỘI XẢY RA HƠN 600 VỤ CHÁY...",810483054558928__0.jpg 810483054558928__1.jp...,post
2,BeatvnNow,810483054558928,https://facebook.com/810483054558928,1.577363e+15,https://facebook.com/1577363486378848,2024-07-02 22:56:48.184952,trong ngõ mà cháy thì khó thoát lắm,810483054558928_1577363486378848_0.jpg 81048...,comment
3,BeatvnNow,810483054558928,https://facebook.com/810483054558928,4.997063e+14,https://facebook.com/499706276053811,2024-07-02 22:56:48.184952,"Kiểu cháy như cơm bữa luôn, sợ thật",810483054558928_499706276053811_0.jpg 810483...,comment
4,BeatvnNow,810023757938191,https://facebook.com/810023757938191,NaN,NaN,2024-07-01 23:51:00,NHÌN LẠI 1 ĐÊM KHÓ QUÊN CÙNG WATERA FESTIVAL 🔥...,810023757938191__0.jpg 810023757938191__1.jp...,post
...,...,...,...,...,...,...,...,...,...
140,BeatvnNow,807300298210537,https://facebook.com/807300298210537,1.168398e+15,https://facebook.com/1168397727646546,2024-06-27 15:58:00,Cặp đôi tham dự ở đây tính như nào zị,807300298210537_1168397727646546_0.jpg,comment
141,BeatvnNow,807300298210537,https://facebook.com/807300298210537,9.966032e+14,https://facebook.com/996603198281046,2024-06-27 15:58:00,Ở đây xịn gấp 10 lần chỗ khác ấy nhỉ,807300298210537_996603198281046_0.jpg,comment
142,BeatvnNow,807300298210537,https://facebook.com/807300298210537,1.887781e+15,https://facebook.com/1887780805017462,2024-06-27 15:58:00,Phải đưa ny đến xem mới được,807300298210537_1887780805017462_0.jpg,comment
143,BeatvnNow,807300298210537,https://facebook.com/807300298210537,4.312911e+14,https://facebook.com/431291133064060,2024-06-27 15:58:00,Miền nam nước pháp này xịn lắm,807300298210537_431291133064060_0.jpg,comment
